## Nielsen Chapter 1 Notebook
### Luke Van Der Male
### https://neuralnetworksanddeeplearning.com/chap1.html

In [9]:
"""
network.py
~~~~~~~~~~

A module to implement the stochastic gradient descent learning
algorithm for a feedforward neural network.  Gradients are calculated
using backpropagation.  Note that I have focused on making the code
simple, easily readable, and easily modifiable.  It is not optimized,
and omits many desirable features.
"""

#### Libraries
# Standard library
import random

# Third-party libraries
import numpy as np

class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print ("Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [10]:
import mnist_loader
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

In [11]:
net = Network([784, 30, 10])

In [12]:
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 9043 / 10000
Epoch 1: 9237 / 10000
Epoch 2: 9284 / 10000
Epoch 3: 9326 / 10000
Epoch 4: 9379 / 10000
Epoch 5: 9372 / 10000
Epoch 6: 9402 / 10000
Epoch 7: 9403 / 10000
Epoch 8: 9437 / 10000
Epoch 9: 9376 / 10000
Epoch 10: 9442 / 10000
Epoch 11: 9465 / 10000
Epoch 12: 9440 / 10000
Epoch 13: 9463 / 10000
Epoch 14: 9455 / 10000
Epoch 15: 9479 / 10000
Epoch 16: 9488 / 10000
Epoch 17: 9499 / 10000
Epoch 18: 9500 / 10000
Epoch 19: 9493 / 10000
Epoch 20: 9491 / 10000
Epoch 21: 9493 / 10000
Epoch 22: 9497 / 10000
Epoch 23: 9501 / 10000
Epoch 24: 9519 / 10000
Epoch 25: 9497 / 10000
Epoch 26: 9495 / 10000
Epoch 27: 9512 / 10000
Epoch 28: 9493 / 10000
Epoch 29: 9512 / 10000


The biases and weights are stored as numpy matrices. So, the following is the weights for the connections between the second and third layer of neurons:

In [13]:
net.weights[1]

array([[ 0.02168052,  3.30403423, -1.20574608, -3.76877688, -2.49152873,
         4.84049761, -5.66634071,  1.13844771, -5.03587812,  0.07432964,
        -2.97537959, -2.80865321,  2.58028027,  6.67081421, -0.356092  ,
        -4.86234495, -1.81287792, -1.94196631, -0.81875337, -2.57625658,
         1.8620166 ,  2.11554511,  0.99838063, -3.52791335, -2.78598009,
        -0.68408559,  1.07599706,  1.31877181, -1.98640913, -3.31610072],
       [ 3.93211628,  0.94642555,  0.15622702,  2.66160413, -1.67861087,
         3.05783072,  1.86405769, -2.73360653,  0.63231235, -3.80876958,
         2.18491922, -8.70054012,  2.92141593, -2.19182498,  2.52880301,
        -2.36826952, -2.41453987, -1.84987684,  1.94761907, -2.11309301,
         3.00989319, -1.96066865,  3.87240739, -1.54415352,  2.35540851,
        -4.41116703,  0.60353547,  1.72379925, -2.04467578,  1.50793735],
       [-6.23303584, -0.99287369,  1.58255898,  1.32316225, -6.66312427,
        -2.32851009,  6.15614868,  3.36831392, -2

The ordering is a little funny. W(j,k) is the kth neuron in the second layer and the jth neuron in the third layer. This is for vectorization.

The sigmoid function is just the python version of Equation (4)

$\sigma(z) = \frac{1}{1 + e^{-z}}$

In [14]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

The feedforward function applies a version of Equation (4) for each layer.

In [15]:
 def feedforward(self, a):
        """Return the output of the network if "a" is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a